In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import signal as sig
import math
from skimage.util import img_as_ubyte, img_as_float64
from skimage.color import rgb2gray
from skimage.color import rgb2hsv


''' Generates guided filter''' 
def guide(I,P,r,e):

    h,w=np.shape(I)
    window = np.ones((r,r))/(r*r)

    meanI = sig.convolve2d(I, window,mode='same')
    meanP = sig.convolve2d(P, window,mode='same')

    corrI = sig.convolve2d(I*I, window,mode='same')
    corrIP = sig.convolve2d(I*P, window,mode='same')

    varI = corrI - meanI*meanI
    covIP = corrIP - meanI*meanP
    a = covIP/(varI+e)
    b = meanP - a*meanI

    meana = sig.convolve2d(a, window,mode='same')
    meanb = sig.convolve2d(b, window,mode='same')

    q = meana*I+meanb

    return q

def localmin(D, r=15):
    R = int(r/2)
    imax = D.shape[0]
    jmax = D.shape[1]
    LM = np.zeros([imax,jmax])
    for i in np.arange(D.shape[0]):
        for j in np.arange(D.shape[1]):
            iL = np.max([i-R,0])
            iR = np.min([i+R, imax])
            jT = np.max([j-R,0])
            jB = np.min([j+R, jmax])
            # print(D[iL:iR+1,jT:jB+1].shape)
            LM[i,j] = np.min(D[iL:iR+1,jT:jB+1])
    return LM

''' It will Apply guided filter to the images and removes haze''' 
def postprocessing(GD, I,V):
    # this will give indices of the columnised image GD
    flat_indices = np.argsort(GD, axis=None)
    R,C = GD.shape
    top_indices_flat = flat_indices[ int(np.round(0.999*R*C)):: ]
    top_indices = np.unravel_index(top_indices_flat, GD.shape)

    max_v_index = np.unravel_index( np.argmax(V[top_indices], axis=None), V.shape )
    I = I/255.0
    A = I[max_v_index[0], max_v_index[1], :]

    beta = 1.0
    transmission = np.minimum( np.maximum(np.exp(-1*beta*GD), 0.1) , 0.9)
    # transmission = np.exp(-1*beta*GD)
    transmission3 = np.zeros(I.shape)
    transmission3[:,:,0] = transmission
    transmission3[:,:,1] = transmission
    transmission3[:,:,2] = transmission

    J = A + (I - A)/transmission3
    J = J - np.min(J)
    J = J/np.max(J)
    return J



def dehaze(img):
    # Read the Image
    #img = cv2.imread("vit.jpg")
    # opencv reads any image in Blue-Green-Red(BGR) format,
    # so change it to RGB format, which is popular.
    I = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Split Image to Hue-Saturation-Value(HSV) format.
    H,S,V = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2HSV) )
    V = V/255.0
    S = S/255.0

    # Calculating Depth Map using the linear model fit by ZHU et al.
    theta_0 = 0.121779
    theta_1 = 0.959710
    theta_2 = -0.780245
    sigma   = 0.041337
    epsilon = np.random.normal(0, sigma, H.shape )
    D = theta_0 + theta_1*V + theta_2*S + epsilon

    # Local Minima of Depth map
    LMD = localmin(D, 15)

    r = 8; # try r=2, 4, 8 or 18
    eps = 0.2 * 0.2; # try eps=0.1^2, 0.2^2, 0.4^2
        # eps *= 255 * 255;   # Because the intensity range of our images is [0, 255]
    GD=guide(D,LMD,r,eps)

    J = postprocessing(GD, I,V)
    return J



def clip(image=None):
    if image is None:
        return None
    image[image < 0] = 0
    image[image > 1] = 1
    return image



def white_balance(img):
    image = img_as_float64(img)

            # Extract colour channels
    R = image[:, :, 2]
    G = image[:, :, 1]
    B = image[:, :, 0]

            # Obtain average intensity for each colour channel
    mean_R = np.mean(R)
    mean_G = np.mean(G)
    mean_B = np.mean(B)

    mean_RGB = np.array([mean_R, mean_G, mean_B])

            # Obtain scaling factor
    grayscale = np.mean(mean_RGB)
    scale = grayscale / mean_RGB

    white_balanced = np.zeros(image.shape)

            # Rescale original intensities
    white_balanced[:, :, 2] = scale[0] * R
    white_balanced[:, :, 1] = scale[1] * G
    white_balanced[:, :, 0] = scale[2] * B

            # Clip to [0.0, 1.0]
    white_balanced = clip(white_balanced)
    return white_balanced


''' equlises the rgb chanells of the image using histograms ''' 
def simplest_cb(image, percent):
    if image is None:
        return None
    out_channels = []
    channels = cv2.split(image)
    totalstop = channels[0].shape[0] * channels[0].shape[1] * percent / 200.0
    for channel in channels:
        bc = cv2.calcHist([channel], [0], None, [256], (0,256), accumulate=False)
        lv = np.searchsorted(np.cumsum(bc), totalstop)
        hv = 255-np.searchsorted(np.cumsum(bc[::-1]), totalstop)
        lut = np.array([0 if i < lv else (255 if i > hv else round(float(i-lv)/float(hv-lv)*255)) for i in np.arange(0, 256)], dtype="uint8")
        out_channels.append(cv2.LUT(channel, lut))
    k = cv2.merge(out_channels)
    #k = cv2.cvtColor(k, cv2.COLOR_BGR2RGB)    
    return k




def luminance_map(image=None):
    if image is None:
        return None
    '''Function to generate the Luminance Weight Map of an image'''
        # Validate parameters
        
    image = img_as_float64(image)

        # Generate Luminance Map
    luminance = np.mean(image, axis=2)
    luminancemap = np.sqrt((1 / 3) * (np.square(image[:, :, 0] - luminance + np.square(image[:, :, 1] - luminance) 
                                                + np.square(image[:, :, 2] - luminance))))
    return luminancemap



def chromatic_map(image=None):
    if image is None:
        return None
    '''Function to generate the Chromatic Weight Map of an image'''
        # Validate parameters
        
    image = img_as_float64(image)
        
        # Convert to HSV colour space
    hsv = rgb2hsv(image)

        # Extract Saturation
    saturation = hsv[:, :, 1]
    max_saturation = 1.0
    sigma = 0.3
        
        # Generate Chromatic Map
    chromaticmap = np.exp(-1 * (((saturation - max_saturation) ** 2) / (2 * (sigma ** 2))))
    return chromaticmap



def saliency_map(image=None):
    if image is None:
        return None
    '''Function to generate the Saliency Weight Map of an image'''
        # Validate parameters   
    image = img_as_float64(image)
        
        # Convert image to grayscale
    if(len(image.shape) > 2):
        image = rgb2gray(image)
    else:
        image = image
        
        # Apply Gaussian Smoothing
    gaussian = cv2.GaussianBlur(image,(5, 5),0) 
        
        # Apply Mean Smoothing
    image_mean = np.mean(image)
        
        # Generate Saliency Map
    saliencymap = np.absolute(gaussian - image_mean)
    return saliencymap


def image_pyramid(image=None, pyramid_type='gaussian', levels=1):
    if image is None:
        return None
        '''Function to generate the Gaussian/Laplacian pyramid of an image'''
        # Validate parameters
    image = img_as_float64(image)
        
        # Generate Gaussian Pyramid
    current_layer = image
    gaussian = [current_layer]
    for i in range(levels):
        current_layer = cv2.pyrDown(current_layer)
        gaussian.append(current_layer)
            
    if pyramid_type == 'gaussian':
        return gaussian
        # Generate Laplacian Pyramid
    elif pyramid_type == 'laplacian':
        current_layer = gaussian[levels-1]
        laplacian = [current_layer]
        for i in range(levels - 1, 0, -1):
            shape = (gaussian[i-1].shape[1], gaussian[i-1].shape[0])
            expand_gaussian = cv2.pyrUp(gaussian[i], dstsize=shape)
            current_layer = cv2.subtract(gaussian[i-1], expand_gaussian)
            laplacian.append(current_layer)
        laplacian.reverse()
        return laplacian

    
    
def weight_maps(J,white_balanced):
    input_images = [
            img_as_float64(J),
            img_as_float64(white_balanced)
        ]

    weight_maps = [
                # Weight maps for first image
                {
                    'luminance': luminance_map(image=input_images[0]),
                    'chromatic': chromatic_map(image=input_images[0]),
                    'saliency': saliency_map(image=input_images[0])
                },

                # Weight maps for second image
                {
                    'luminance': luminance_map(image=input_images[1]),
                    'chromatic': chromatic_map(image=input_images[1]),
                    'saliency': saliency_map(image=input_images[1])
                }
            ]
    weight_maps[0]['combined'] = (weight_maps[0]['luminance'] * weight_maps[0]['chromatic'] * weight_maps[0]['saliency'])
    weight_maps[1]['combined'] = (weight_maps[1]['luminance'] * weight_maps[1]['chromatic'] * weight_maps[1]['saliency'])

            # Normalized weight maps
    weight_maps[0]['normalized'] = weight_maps[0]['combined'] / (weight_maps[0]['combined'] + weight_maps[1]['combined'])
    weight_maps[1]['normalized'] = weight_maps[1]['combined'] / (weight_maps[0]['combined'] + weight_maps[1]['combined'])
    
    pyramid_height=12
    gaussians = [
            image_pyramid(image=weight_maps[0]['normalized'], pyramid_type='gaussian', levels=pyramid_height),
            image_pyramid(image=weight_maps[1]['normalized'], pyramid_type='gaussian', levels=pyramid_height)
        ]
    l=[input_images,weight_maps,gaussians]
    return l




def fusion(inputs=None, weights=None, gaussians=None):
    if inputs is None or weights is None or gaussians is None:
        return None
        '''Function to fuse the pyramids together'''
        # Validate parameters
    fused_levels = []

        # Perform Fusion by combining the Laplacian and Gaussian pyramids
    for i in range(len(gaussians[0])):
        if len(inputs[0].shape) > 2:
            for j in range(inputs[0].shape[2]):
                laplacians = [
                        image_pyramid(image=inputs[0][:, :, j], pyramid_type='laplacian', levels=len(gaussians[0])),
                        image_pyramid(image=inputs[1][:, :, j], pyramid_type='laplacian', levels=len(gaussians[0]))
                             ]
                    # Adjust rows to match
                row_size = np.min(np.array([
                        laplacians[0][i].shape[0],
                        laplacians[1][i].shape[0],
                        gaussians[0][i].shape[0],
                        gaussians[1][i].shape[0]
                    ]))

                    # Adjust columns to match
                col_size = np.min(np.array([
                        laplacians[0][i].shape[1],
                        laplacians[1][i].shape[1],
                        gaussians[0][i].shape[1],
                        gaussians[1][i].shape[1]
                    ]))
                    
                if j == 0:
                    intermediate = np.ones(inputs[0][:row_size, :col_size].shape)
                    # Fusion Step
                intermediate[:, :, j] = (laplacians[0][i][:row_size, :col_size] * gaussians[0][i][:row_size, :col_size]) + (laplacians[1][i][:row_size, :col_size] * gaussians[1][i][:row_size, :col_size])
            fused_levels.append(intermediate)
        
        # Reconstruct Image Pyramids
    for i in range(len(fused_levels)-2, -1, -1):
        level_1 = cv2.pyrUp(fused_levels[i+1])
        level_2 = fused_levels[i]
        r = min(level_1.shape[0], level_2.shape[0])
        c = min(level_1.shape[1], level_2.shape[1])
        fused_levels[i] = level_1[:r, :c] + level_2[:r, :c]

        # Clip fused image to [0.0, 1.0]
    fused = clip(fused_levels[0])
    return fused


def enhance_contrast(image=None):
        '''Function to enhance contrast in an image'''
        # Validate parameters
        if image is None:
            return None

        image = img_as_float64(image)

        # Extract colour channels
        R = image[:, :, 2]
        G = image[:, :, 1]
        B = image[:, :, 0]

        # Obtain luminance using predefined scale factors
        luminance = 0.299 * R + 0.587 * G + 0.114 * B
        mean_luminance = np.mean(luminance)

        # Compute scale factor
        gamma = 2 * (0.5 + mean_luminance)

        # Scale mean-luminance subtracted colour chanels 
        enhanced = np.zeros(image.shape)
        enhanced[:, :, 2] = gamma * (R - mean_luminance)
        enhanced[:, :, 1] = gamma * (G - mean_luminance)
        enhanced[:, :, 0] = gamma * (B - mean_luminance)

        # Clip to [0.0, 1.0]
        enhanced = clip(enhanced)

        return enhanced
